In [20]:
from sympy.matrices import Matrix
from sympy import Symbol
import sympy as sp
from sympy.printing.fortran import fcode
import sympy.printing as printing


In [21]:
def rank_of_temporaries(rvar):

    t_rank = len(rvar)*[0]

    for i, (var, var_expr) in enumerate(rvar):

        has_dx1 = False
        has_dx2 = False
        has_ddx = False
        if "_dx1" in str(var_expr):
            has_dx1 = True
        if "_dx2" in str(var_expr):
            has_dx2 = True
        if "_ddx" in str(var_expr):
            has_ddx = True

        if has_dx1 and has_dx2:
            t_rank[i] = 2
        elif has_ddx:
            t_rank[i] = 2
        elif has_dx1 or has_dx2:
            t_rank[i] = 1
        else:
            t_rank[i] = 0

        for j, (varj, var_exprj) in enumerate(rvar[0:i]):
            if str(varj) in str(var_expr):
                t_rank[i] = max(t_rank[i], t_rank[j])

    return t_rank

In [27]:

u, v = sp.symbols('u v')
du1, dv1 = sp.symbols('u_dx1 v_dx1')
du2, dv2 = sp.symbols('u_dx2 v_dx2')
ddu, ddv = sp.symbols('u_ddx v_ddx')
x = Matrix([u, v])
dx1 = Matrix([du1, dv1])
dx2 = Matrix([du2, dv2])
ddx = Matrix([ddu, ddv])

narg = 2
def fun(u, v):
    return u/v

# narg = 1
# def fun(u):
#     return sp.log(u)

if narg == 1:
    f = fun(u)
    df_dx = sp.diff(f, u)
    df1 = df_dx*du1
    ddf = du1*sp.diff(sp.diff(f, u), u)*du2 + sp.diff(f, u)*ddu
elif narg == 2:
    f = fun(u, v)
    df_dx = sp.diff(f, x)
    df1 = (df_dx.T*dx1)[0]
    ddf = (dx1.T*sp.hessian(f, x)*dx2 + sp.diff(f, x).T*ddx)[0]

expr = (f, df1, ddf)
assign_to = ('f', 'df', 'ddf')

rvar, rexpr = sp.cse(sp.simplify(expr),
    order='none', list=False, symbols=sp.utilities.iterables.numbered_symbols(prefix='t', start=0), 
    ignore=(du1, du2, ddu, dv1, dv2, ddv), optimizations='basic')

t_rank = rank_of_temporaries(rvar)



# out = []

# for var, var_expr in rvar:
#     outi = fcode(var_expr,assign_to=var,source_format="free",standard=95)
#     out.append(outi)

# for var, var_expr in zip(assign_to,rexpr):
#     outi = fcode(var_expr,assign_to=var,source_format="free",standard=95)
#     out.append(outi)


# for o in out:
#     print(o)


In [28]:
print("Function value: ")
r = 0
for i, (var, var_expr) in enumerate(rvar):
    if t_rank[i] == r:
        print(fcode(var_expr,assign_to=var,source_format="free",standard=95))
print(fcode(rexpr[r],assign_to=assign_to[r],source_format="free",standard=95))

print("First derivative: ")
r=1
for i, (var, var_expr) in enumerate(rvar):
    if t_rank[i] == r:
        print(fcode(var_expr,assign_to=var,source_format="free",standard=95))
print(fcode(rexpr[r],assign_to=assign_to[r],source_format="free",standard=95))

r=2
for i, (var, var_expr) in enumerate(rvar):
    if t_rank[i] == r:
        print(fcode(var_expr,assign_to=var,source_format="free",standard=95))
print(fcode(rexpr[r],assign_to=assign_to[r],source_format="free",standard=95))

Function value: 
t0 = 1d0/v
t1 = t0*u
f = t1
First derivative: 
df = t0*(-t1*v_dx1 + u_dx1)
ddf = t0*(-t0*u_dx2*v_dx1 + t0*v_dx2*(2*t1*v_dx1 - u_dx1) - t1*v_ddx + &
      u_ddx)


In [29]:
print(rvar)

[(t0, 1/v), (t1, t0*u)]


In [30]:
print(rexpr)

(t1, t0*(-t1*v_dx1 + u_dx1), t0*(-t0*u_dx2*v_dx1 + t0*v_dx2*(2*t1*v_dx1 - u_dx1) - t1*v_ddx + u_ddx))


In [31]:
print(t_rank)

[0, 0]
